Документация к Telegram-боту для продажи курсов через CRM Bitrix24

Описание

Этот бот позволяет пользователям выбирать курсы, тарифы и отправлять данные в CRM Bitrix24. Также реализована реферальная система.

Основные библиотеки

In [ ]:
import telebot
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton
import requests
from refral import TOKEN ,BITRIX_WEBHOOK_URL

telebot — используется для работы с Telegram API.
requests — для отправки HTTP-запросов в Bitrix24.
TOKEN и BITRIX_WEBHOOK_URL импортируются из refral.py и содержат API-ключ бота и вебхук CRM.



Глобальные переменные

In [ ]:
bot = telebot.TeleBot(TOKEN)
user_states = {}
referrals = {}

bot — объект бота.

user_states — словарь для хранения данных пользователей.

referrals — словарь для хранения количества приглашенных пользователей

Список курсов, доступных для покупки.

In [ ]:
courses = {
    "course1": "Wileberiser",
    "course2": "OZON",
    "course3": "AVITO",
    "course4": "YANDEX MARKAT"
}

Список тарифов с ценами.

In [ ]:
v=tariffs = {
    "tariff1": {"name": "Основной", "price": 500},
    "tariff2": {"name": "Стандарт", "price": 1000},
    "tariff3": {"name": "Премиум", "price": 1500},
    "tariff4": {"name": "VIP", "price": 2000}
}

Функции для работы с рефералам

In [ ]:
def generate_referral_link(user_id):
    return f"https://t.me/your_bot?start=ref{user_id}

Генерирует реферальную ссылку для пользователя.

Функции создания клавиатур

In [ ]:
def create_courses_keyboard():
    keyboard = InlineKeyboardMarkup()
    for course_id, course_name in courses.items():
        keyboard.add(InlineKeyboardButton(course_name, callback_data=f"course_{course_id}"))
    return keyboard

Создает клавиатуру с курсами.

In [ ]:
def create_payment_method_keyboard():
    keyboard = InlineKeyboardMarkup()
    keyboard.row(
        InlineKeyboardButton("Онлайн", callback_data="payment_online"),
        InlineKeyboardButton("Менеджер", callback_data="payment_manager")
    )
    return keyboard

Клавиатура выбора метода оплаты.

Создает клавиатуру с тарифами.

In [ ]:
def create_tariffs_keyboard(course_id):
    keyboard = InlineKeyboardMarkup()
    for tariff_id, tariff_info in tariffs.items():
        button_text = f"{tariff_info['name']} - {tariff_info['price']} рубл"
        keyboard.add(InlineKeyboardButton(button_text, callback_data=f"tariff_{course_id}_{tariff_id}"))
    return keyboard



Обработчики команд

/start — запуск бота

In [ ]:
@bot.message_handler(commands=["start"])
def start_handler(message):
    user_id = message.from_user.id
    args = message.text.split()
    
    if len(args) > 1 and args[1].startswith("ref"):
        referrer_id = args[1][3:]
        if referrer_id.isdigit():
            referrer_id = int(referrer_id)
            if referrer_id != user_id:
                referrals.setdefault(referrer_id, 0)
                referrals[referrer_id] += 1

    referral_link = generate_referral_link(user_id)
    user_states[user_id] = {"referral_link": referral_link}

    keyboard = telebot.types.ReplyKeyboardMarkup(row_width=1, resize_keyboard=True)
    button_phone = telebot.types.KeyboardButton(text="Отправьте свой номер телефона", request_contact=True)
    keyboard.add(button_phone)

    bot.send_message(
        message.chat.id,
        "Привет! Пожалуйста, отправьте свой номер телефона:",
        reply_markup=keyboard
    )

-Запускает бота и создает реферальную ссылку.

-Проверяет, был ли пользователь приглашен.

-Запрашивает номер телефона.

Получение номера телефона

Сохраняет номер телефона и предлагает выбрать курс.

In [ ]:
@bot.message_handler(content_types=["contact"])
def contact_handler(message):
    user_id = message.from_user.id
    
    if user_id not in user_states:
        user_states[user_id] = {}
    
    user_states[user_id]["phone"] = message.contact.phone_number

    bot.send_message(
        message.chat.id,
        "Пожалуйста, выберите курс:",
        reply_markup=create_courses_keyboard()
    )



Выбор курса

Сохраняет выбранный курс и предлагает выбрать метод оплаты.

In [ ]:
@bot.callback_query_handler(func=lambda call: call.data.startswith("course_"))
def course_callback(call):
    user_id = call.from_user.id
    course_id = call.data.split("_")[1]
    user_states[user_id]["selected_course"] = course_id

    bot.edit_message_text(
        " Выберите метод оплаты:",
        call.message.chat.id,
        call.message.message_id,
        reply_markup=create_payment_method_keyboard()
    )



Выбор метода оплаты

Если пользователь выбирает «Менеджер», отправляет контакт.

Если выбирает «Онлайн», предлагает тарифы.

In [ ]:
@bot.callback_query_handler(func=lambda call: call.data.startswith("payment_"))
def payment_method_callback(call):
    user_id = call.from_user.id
    payment_method = call.data.split("_")[1]

    if payment_method == "manager":
        bot.edit_message_text(
            " Свяжитесь с менеджером: @unknownnnsu",
            call.message.chat.id,
            call.message.message_id
        )
    else:
        course_id = user_states[user_id]["selected_course"]
        bot.edit_message_text(
            "Пожалуйста, выберите тариф:",
            call.message.chat.id,
            call.message.message_id,
            reply_markup=create_tariffs_keyboard(course_id)
        )

Отправка данных в Bitrix24

Отправляет данные сделки в CRM Bitrix24.

In [ ]:
response = requests.post(f"{BITRIX_WEBHOOK_URL}/crm.deal.add", json=deal_data)

Запуск бота
Запускает бота на постоянную работу.

In [ ]:
bot.polling(none_stop=True)